# Initial data exploration

Let's see what we have in the database:

In [81]:
# The number of uniq. values for given key in database
from pymongo import MongoClient
from tabulate import tabulate


db = MongoClient()['stores']
col = db.data

TOTAL_NUMBER_OF_PRODUCTS = col.count()

db_size = col.count()
keys = ['brand', 'category', 'size', 'color', 'code']
uniq_values_by_key = []
for key in keys:
    res = col.distinct(key)
    items_per_key = round(db_size / len(res))
    uniq_values_by_key.append((key, len(res), items_per_key))

print(tabulate(uniq_values_by_key, headers=['Key', 'Uniq. Values', 'Avg. items per key'], tablefmt="orgtbl"))
print("Products in DB: {}".format(TOTAL_NUMBER_OF_PRODUCTS))

| Key      |   Uniq. Values |   Avg. items per key |
|----------+----------------+----------------------|
| brand    |            338 |                 4619 |
| category |            527 |                 2962 |
| size     |           1117 |                 1398 |
| color    |          20333 |                   77 |
| code     |          20078 |                   78 |
Products in DB: 1561159


## Products per Distributor

In [36]:
DISTRIBUTORS = list(col.distinct("source"))
results = col.aggregate(
    [
        {
            "$group": {
                "_id": "$source",
                "count": {"$sum": 1},
            }
        },
        {
            "$sort": {
                "count": -1,
            }
        }
    ]
)
PRODUCTS_PER_DISTRIBUTOR = [
    (str(x['_id']), x['count']) 
    for x in list(results)
]
print(tabulate(PRODUCTS_PER_DISTRIBUTOR, 
               headers=['Distributor', 'Number of Products'], 
               tablefmt="orgtbl"))

| Distributor   |   Number of Products |
|---------------+----------------------|
| marsha        |               226515 |
| ignacia       |               209310 |
| jeneva        |               145671 |
| patti         |               130398 |
| shemeka       |               126732 |
| evie          |               109834 |
| larry         |               105068 |
| kina          |                88768 |
| allegra       |                76808 |
| major         |                65977 |
| titus         |                60460 |
| rossana       |                49008 |
| denisha       |                45662 |
| carey         |                36457 |
| rubi          |                34641 |
| wendell       |                25784 |
| joanie        |                10345 |
| yolande       |                 6388 |
| zack          |                 3715 |
| ciara         |                 3618 |


## Top Brands

In [37]:
results = col.aggregate(
    [
        {
            "$group": {
                "_id": "$brand",
                "count": {"$sum": 1},
            }
        },
        {
            "$sort": {
                "count": -1,
            }
        },
        {
            "$limit": 20
        }
    ]
)
TOP_BRANDS = [(str(x['_id']), x['count']) for x in list(results)]
print(tabulate(TOP_BRANDS, 
               headers=['Brand', 'Number of Products'], tablefmt="orgtbl"))


| Brand             |   Number of Products |
|-------------------+----------------------|
| augusta           |               137980 |
|                   |               126929 |
| gildan            |                94038 |
| badger            |                72739 |
| None              |                64153 |
| port authority    |                53788 |
| hanes             |                50641 |
| comfort colors    |                42246 |
| jerzees           |                34338 |
| next level        |                33791 |
| anvil             |                32603 |
| bella+canvas      |                32241 |
| ultraclub         |                29842 |
| augusta drop ship |                28439 |
| port & company    |                28186 |
| champion          |                24737 |
| sport-tek         |                24373 |
| bella + canvas    |                24037 |
| badger sport      |                23863 |
| a4                |                23769 |


Brand field is empty for 126929 + 64153 products.

## Top Sizes


In [38]:
results = col.aggregate(
    [
        {
            "$group": {
                "_id": "$size",
                "count": {"$sum": 1},
            }
        },
        {
            "$sort": {
                "count": -1,
            }
        },
        {
            "$limit": 20
        }
    ]
)
TOP_SIZES = [(str(x['_id']), x['count']) for x in list(results)]
print(tabulate(TOP_SIZES, headers=['Size', 'Number of Products'], tablefmt="orgtbl"))

| Size     |   Number of Products |
|----------+----------------------|
| M        |               202965 |
| L        |               202641 |
| S        |               202589 |
| XL       |               186166 |
| 2XL      |               152014 |
| 3XL      |               106966 |
| XS       |                71104 |
| 4XL      |                50869 |
| MEDIUM   |                31645 |
| LARGE    |                31599 |
| SMALL    |                31506 |
| EXTRA    |                31098 |
| 2X       |                20091 |
| 3X       |                20085 |
| 5XL      |                17541 |
| ONE      |                13786 |
| XXL      |                11288 |
| OS       |                 7615 |
| ONE SIZE |                 7269 |
| XX       |                 7184 |


# Top Categories

In [39]:
results = col.aggregate(
    [
        {
            "$group": {
                "_id": "$category",
                "count": {"$sum": 1},
            }
        },
        {
            "$sort": {
                "count": -1,
            }
        },
        {
            "$limit": 20
        }
    ]
)
TOP_CATEGORIES = [(str(x['_id']), x['count']) for x in list(results)]
print(tabulate(TOP_CATEGORIES, headers=['Category', 'Number of Products'], tablefmt="orgtbl"))

| Category             |   Number of Products |
|----------------------+----------------------|
| t-shirts             |               193663 |
| None                 |               129013 |
| activewear           |                50966 |
| tee shirts           |                43741 |
| augusta              |                35186 |
| mens tees            |                33545 |
| performance          |                29677 |
| teamwear             |                29162 |
| augusta drop ship    |                28439 |
| fleece               |                27280 |
| polos                |                25884 |
| outerwear            |                25068 |
| polos/knits          |                24863 |
| t shirts             |                24755 |
| [mens]               |                23350 |
| gildan               |                19964 |
| big & tall+          |                19377 |
| badger               |                18021 |
| womens tees          |                

## Top Colors

In [44]:
results = col.aggregate(
    [
        {
            "$group": {
                "_id": "$color",
                "count": {"$sum": 1},
            }
        },
        {
            "$sort": {
                "count": -1,
            }
        },
        {
            "$limit": 20
        }
    ]
)
TOP_COLORS = [(str(x['_id']), x['count']) for x in list(results)]
print(tabulate(TOP_COLORS, headers=['Color', 'Number of Products'], tablefmt="orgtbl"))

| Color        |   Number of Products |
|--------------+----------------------|
| BLACK        |                95738 |
| WHITE        |                68948 |
| NAVY         |                57694 |
| RED          |                42204 |
| ROYAL        |                36039 |
| MAROON       |                22059 |
| PURPLE       |                19195 |
| CHARCOAL     |                16196 |
| GOLD         |                13800 |
| LIGHT BLUE   |                12964 |
| ORANGE       |                12878 |
| FOREST GREEN |                12213 |
| GRAPHITE     |                 9854 |
| SILVER       |                 9165 |
| ASH          |                 9017 |
| PINK         |                 7624 |
| FOREST       |                 7570 |
| LIME         |                 7537 |
| KELLY        |                 7487 |
| BLACK/WHITE  |                 7391 |


## Top Codes

In [43]:
results = col.aggregate(
    [
        {
            "$group": {
                "_id": "$code",
                "count": {"$sum": 1},
            }
        },
        {
            "$sort": {
                "count": -1,
            }
        },
        {
            "$limit": 20
        }
    ]
)
TOP_CODES = [(str(x['_id']), x['count']) for x in list(results)]
print(tabulate(TOP_CODES, headers=['Code', 'Number of Products'], tablefmt="orgtbl"))

| Code   |   Number of items |
|--------+-------------------|
| 874    |              3219 |
| 5180   |              3098 |
| 2000   |              3060 |
| 5250   |              2910 |
| 5000   |              2801 |
| 29M    |              2764 |
| 980    |              2521 |
| 5170   |              2392 |
| 4400   |              2329 |
| 3001   |              2328 |
| 1717   |              2304 |
| 3005   |              2280 |
| 3001C  |              2224 |
| 4980   |              2200 |
| 3480   |              2063 |
| 6004   |              2055 |
| M1009  |              2002 |
| 29B    |              1943 |
| 3600   |              1906 |
| 9030   |              1857 |


## Measurements per Distributor

Collect the number of uniq brands, color, sizes, categories and codes for each distributor

In [52]:
MEASUREMENT_PER_DISTRIBUTOR = col.aggregate(
    [
        {
            "$group": {
                "_id": "$source",
                "brands": {"$addToSet": "$brand"},
                "colors": {"$addToSet": "$color"},
                "sizes": {"$addToSet": "$size"},
                "categories": {"$addToSet": "$category"},
                "codes": {"$addToSet": "$code"}
            }
        },
        {
            "$project": {
                "_id": 1,
                "brands": {"$size": "$brands"},
                "colors": {"$size": "$colors"},
                "sizes": {"$size": "$sizes"},
                "categories": {"$size": "$categories"},
                "codes": {"$size": "$codes"},
            }
        },
        {
            "$sort": {
                "categories": -1,
                "brands": -1,
                "colors": -1,
                "sizes": -1,
                "codes": -1,
            }
        },
    ]
)

TABLE_MEASUREMENTS_PER_DISTRIBUTOR = [
    (str(x['_id']), x['brands'], x['colors'], 
     x['sizes'], x['categories'], x['codes']) 
    for x in list(MEASUREMENT_PER_DISTRIBUTOR)
]
print(tabulate(TABLE_MEASUREMENTS_PER_DISTRIBUTOR, 
               headers=[
                   'Distributor', 'Brands', 'Colors', 
                   'Sizes', 'Categories', 'Codes'], 
               tablefmt="orgtbl"))



| Distributor   |   Brands |   Colors |   Sizes |   Categories |   Codes |
|---------------+----------+----------+---------+--------------+---------|
| marsha        |      123 |     6715 |     222 |          143 |    5210 |
| ignacia       |      114 |     6076 |     106 |          114 |    5116 |
| shemeka       |       33 |     3385 |     286 |           13 |    2055 |
| jeneva        |       66 |     3011 |     491 |           66 |    2675 |
| denisha       |        1 |     2746 |      74 |           34 |     885 |
| patti         |       58 |     2687 |      81 |           26 |    3054 |
| larry         |       72 |     2600 |     107 |            8 |    2345 |
| evie          |       73 |     2515 |      64 |           20 |    2539 |
| kina          |       40 |     2295 |     189 |           29 |    1927 |
| titus         |       78 |     2148 |      50 |            7 |    1370 |
| rubi          |       10 |     1697 |     186 |           54 |    1666 |
| allegra       |        

## Plot joined bar chat

In [82]:
import numpy as np
from plotly.offline import iplot, init_notebook_mode
from plotly.graph_objs import Bar
init_notebook_mode(connected=True)


def get_bar(df, name, color):
    return Bar(
        x=['Brnd', 'Clrs', 'Szs', 'Cts', 'Cds'], y=df[1:], 
        name=name, marker=dict(color=color))

data = []
colors = ['#ffcdd2', '#a2d5f2', '#235412', '#415220', '#59606d']
for dist in TABLE_MEASUREMENTS_PER_DISTRIBUTOR:
    name = dist[0]
    data.append(get_bar(dist, name, str(iter(colors))))

layout = Layout(title="Joined chart",
                xaxis=dict(title='Measurement'),
                yaxis=dict(title='Products'))
fig = Figure(data=data, layout=layout)
iplot(fig, filename='jupyter/joined_chart')


Looking on chart above easy i guess i'll start with sizes. categories and brands.